If you need a Computer Vision API account, you can create one with this Azure CLI command:

RES_REGION=westus2
RES_GROUP=<resourcegroup-name>
ACCT_NAME=<computervision-account-name>

az cognitiveservices account create \
    --resource-group $RES_GROUP \
    --name $ACCT_NAME \
    --location $RES_REGION \
    --kind ComputerVision \
    --sku S1 \
    --yes

Setup Environment

prerequisites: 
pip install azure-cognitiveservices-vision-computervision

In [4]:
import os
import sys
import requests
from io import BytesIO

setup 2 env variables:

RES_GROUP=<resourcegroup-name>
ACCT_NAME=<computervision-account-name>

export ACCOUNT_REGION=$(az cognitiveservices account show \
    --resource-group $RES_GROUP \
    --name $ACCT_NAME \
    --query location \
    --output tsv)

export ACCOUNT_KEY=$(az cognitiveservices account keys list \
    --resource-group $RES_GROUP \
    --name $ACCT_NAME \
    --query key1 \
    --output tsv)

In [5]:
os.environ["ACCOUNT_REGION"] = ""
os.environ["ACCOUNT_KEY"] = ""

In [6]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

import os
region = os.environ['ACCOUNT_REGION']
key = os.environ['ACCOUNT_KEY']

credentials = CognitiveServicesCredentials(key)
client = ComputerVisionClient(
    endpoint="https://domain-computer-vision-dfs.cognitiveservices.azure.com/",
    credentials=credentials
)

In [7]:
# import models
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
import time

url = "https://github.com/Azure-Samples/cognitive-services-python-sdk-samples/raw/master/samples/vision/images/make_things_happen.jpg"
raw = True
numberOfCharsInOperationId = 36

# SDK call
rawHttpResponse = client.read(url, language="en", raw=True)

# Get ID from returned headers
operationLocation = rawHttpResponse.headers["Operation-Location"]
idLocation = len(operationLocation) - numberOfCharsInOperationId
operationId = operationLocation[idLocation:]

# SDK call
while True:
    result = client.get_read_result(operationId)
    if result.status not in ['notStarted', 'running']:
        break
    time.sleep(1)


In [8]:
result.status

<OperationStatusCodes.succeeded: 'succeeded'>

In [9]:
# Get data
if result.status == OperationStatusCodes.succeeded:

    for line in result.analyze_result.read_results[0].lines:
        print(line.text)
        # print(line.bounding_box)

make
things
happen
